In [1]:
import os, time, shlex, subprocess

import GPUtil, psutil


def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

class UEX:
    cmd = []
    wd = ""
    n = 0
    process = None
    ueprocess = None

    def __int__(self, cmd: str) -> bool:
        x = shlex.split(cmd)
        if len(x) < 2: return False
        self.cmd, self.wd = x, os.getcwd()


    def run(self) -> bool:
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        poll = process.poll()
        if poll is None: return False

        for proc in psutil.process_iter():
            if proc.name() == self.cmd[1] and len(proc.children()) == 0:
                self.ueprocess = proc
                break

        self.process = process
        self.n = 1
        return True


    def cpuInfo() -> str:
        cpu_freq = psutil.cpu_freq()
        return (f"{'='*20} CPU Info {'='*20}\n"              \
        f"Physical cores: psutil.cpu_count(logical=False)\n" \
        f"Total cores: psutil.cpu_count(logical=True)\n"     \
        f"Max Frequency: {cpu_freq.max:.2f}MhzMhz\n"      \
        f"Min Frequency: {cpu_freq.min:.2f}Mhz\n"         \
        f"Current Frequency: {cpu_freq.current:.2f}Mhz\n" \
        f"Total CPU Usage: {psutil.cpu_percent()}%")


    def memInfo() -> str:
        svmem = psutil.virtual_memory()
        swap = psutil.swap_memory()

        return (f"{'='*20} Memory Information {'='*20}\n" \
        f"Total: {get_size(svmem.total)}\n"               \
        f"Available: {get_size(svmem.available)}\n"       \
        f"Used: {get_size(svmem.used)}\n"  \
        f"Percentage: {svmem.percent}%\n"  \
        f"{'='*20}  SWAP {'='*20}\n"       \
        f"Total: {get_size(swap.total)}\n" \
        f"Free: {get_size(swap.free)}\n"   \
        f"Used: {get_size(swap.used)}\n"   \
        f"Percentage: {swap.percent}%\n")

    def gpuInfo() -> str:
        text = f"{'='*40} GPU Details {'='*40}\n"
        gpus = GPUtil.getGPUs()
        list_gpus = []
        for gpu in gpus:
            gpu_id = gpu.id
            gpu_name = gpu.name
            gpu_load = f"{gpu.load*100}%"
            gpu_free_memory = f"{gpu.memoryFree}MB"
            gpu_used_memory = f"{gpu.memoryUsed}MB"
            gpu_total_memory = f"{gpu.memoryTotal}MB"
            gpu_temperature = f"{gpu.temperature} °C"
            gpu_uuid = gpu.uuid

            list_gpus.append((
              gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
              gpu_total_memory, gpu_temperature, gpu_uuid
            ))

        return text + tabulate(list_gpus, headers=("id", "name", "load", "free memory",
            "used memory", "total memory", "temperature", "uuid"))

In [6]:
UEX("x a c d")

TypeError: UEX() takes no arguments